## Calibration interrogation
Examine the parameter outputs from the calibration algorithm.

In [ ]:
import numpy as np
import arviz as az

from aust_covid.inputs import get_ifrs
from aust_covid.calibration import get_priors
from emutools.utils import load_param_info
from emutools.calibration import plot_param_progression, plot_posterior_comparison, tabulate_calib_results
from aust_covid.constants import RUN_IDS, RUNS_PATH, PRIMARY_ANALYSIS, BURN_IN
from emutools.tex import DummyTexDoc

In [ ]:
param_info = load_param_info()
parameters = param_info['value'].to_dict() | get_ifrs(DummyTexDoc())
priors = get_priors(PRIMARY_ANALYSIS in ['vacc', 'both'], param_info['abbreviations'], DummyTexDoc())
prior_names = [p.name for p in priors]
n_half_priors = round(len(priors) / 2)
first_priors = prior_names[:n_half_priors]
last_priors = prior_names[n_half_priors:]
first_third_priors = prior_names[:6]
middle_third_priors = prior_names[6: 12]
last_third_priors = prior_names[12:]

In [ ]:
idata = az.from_netcdf(RUNS_PATH / RUN_IDS[PRIMARY_ANALYSIS] / 'output/calib_full_out.nc')
idata = idata.sel(draw=np.s_[BURN_IN:])

### Parameter traces with estimated posterior densities by chain
First of three parameter groups traces.

In [ ]:
plot_param_progression(idata, param_info['descriptions'], req_vars=first_third_priors)

Second of three parameter groups traces.

In [ ]:
plot_param_progression(idata, param_info['descriptions'], req_vars=middle_third_priors)

Third of three parameter groups traces.

In [ ]:
plot_param_progression(idata, param_info['descriptions'], req_vars=last_third_priors)

### Comparison of parameter posteriors to priors
First of two parameter groups distributions.

In [ ]:
plot_posterior_comparison(idata, priors, first_priors, param_info['abbreviations'].to_dict(), 0.995)

Second of two parameter groups distributions.

In [ ]:
plot_posterior_comparison(idata, priors, last_priors, param_info['abbreviations'].to_dict(), 0.995)

### Tabular results of parameter distributions

In [ ]:
tabulate_calib_results(idata, priors, param_info)